In [1]:
"""Load the primitive model and the DSL parse"""
from karanir.thanagor.model import *
from karanir.thanagor import KaranirThanagor, config

[Taichi] version 1.7.0, llvm 15.0.7, commit 7b58b0ff, osx, python 3.9.13


[I 01/13/24 16:25:32.179 4263916] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
"""Define the domain specification for the demo test case"""
from karanir.thanagor.domain import load_domain_string
demo_domain_string = f"""
(domain scourge_demo)
(:type
    object - vector[float,100]
    position - vector[float,2]
    color - vector[float, 64]
    category
)
(:predicate
    color ?x-object -> vector[float,100]
    shape ?x-object -> vector[float,100]
    is-red ?x-object -> boolean
    is-blue ?x-object -> boolean
    is-ship ?x-object -> boolean
    is-house ?x-object -> boolean
)
(:derived
    is-green ?x-color expr: (??f ?x)

    is-good ?x ?y 
    expr:   (or (??f ?x) (??g ?x))
)
(:constraint
    (color: is-red is-blue)
    (category: is-ship is-house)
)
(:action
    (
        name:pick-up
        parameters:?x-object ?y
        precondition: (is-red (color ?x))
        effect: (
            if (is-blue ?y)
                (and
                (assign (is-blue ?x) 1.0 )
                (assign (is-red ?y) 0.0)
                )
            )
    )
)
"""
domain = load_domain_string(demo_domain_string)
domain.print_summary()

domain:
  scourge_demo
types:
  object - vector[float,['100']]
  position - vector[float,['2']]
  color - vector[float,['64']]
  category - object
predicates:
  color:['?x-object'] -> vector[float,['100']]
  shape:['?x-object'] -> vector[float,['100']]
  is-red:['?x-object'] -> boolean
  is-blue:['?x-object'] -> boolean
  is-ship:['?x-object'] -> boolean
  is-house:['?x-object'] -> boolean
actions:
 name:pick-up
  params:['?x-object', '?y']
  precond:(is-red (color ?x))
  effect:(if (is-blue ?y) (and (assign (is-blue ?x) 1.0) (assign (is-red ?y) 0.0)))
{'is-green-f': None, 'is-good-f': None, 'is-good-g': None}


In [3]:
from karanir.thanagor.knowledge.executor import CentralExecutor
from karanir.thanagor.model import config, KaranirThanagor

model = KaranirThanagor(domain, config)
model.central_executor.auto_fillup()
#model.print_summary()
print(model.central_executor.predicates)
print(model.central_executor.derived)

arity: 1
color
shape
is-red
is-blue
is-ship
is-house
{1: [color, shape, is-red, is-blue, is-ship, is-house]}
{'is-green': {'parameters': ['?x-color'], 'expr': '(is-green-f ?x)'}, 'is-good': {'parameters': ['?x', '?y'], 'expr': '(or (is-good-f ?x) (is-good-g ?x))'}}


In [15]:
from karanir.thanagor.program import Program
from karanir.thanagor.dsl.vqa_primitives import Boolean
from karanir.utils.tensor import stats_summary
# [{'params': model.parameters()}]
print(Primitive.GLOBALS)

class newGetter(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Parameter(torch.tensor(1.0))

class ShapeMapper(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(132,100,1,1)
    def forward(self, x):
        if len(x.shape) == 3: x = x.unsqueeze(0)
        x = x.permute(0,3,1,2)
        return self.conv(x).permute(0,2,3,1)[0]

color_mapper = nn.Linear(132,100)
shape_mapper = ShapeMapper()

Primitive("new_func",Boolean, newGetter())
color = Primitive.GLOBALS["color"]
color.value = lambda x: {"features":color_mapper(x["features"]), "executor":x["executor"]}

shape = Primitive.GLOBALS["shape"]
shape.value = lambda x:{"features":

    shape_mapper(x["features"]), "executor":x["executor"]}

def gather_parameters(model = None):
    params = []
    if model is not None: params.append({"model_params":model.parameters()})
    for prim in Primitive.GLOBALS:
        if isinstance(Primitive.GLOBALS[prim].value,nn.Module):
            param = Primitive.GLOBALS[prim].value.parameters()
            params.append({"param":param})
    return params

params = gather_parameters(model)

features = torch.randn([5,7,132])
end = torch.ones([5,7,1])
context = {
    "features":features,
    "end":end,
    "executor":model.central_executor}
output = model.central_executor.evaluate("(Pr (color $0) is-red)",{0:context})

stats_summary(output["logits"])

output = model.central_executor.evaluate("(Pr (shape $0) is-house)",{0:context})

stats_summary(output["logits"])


{'exists': exists, 'filter': filter, 'filter_expr()': filter_expr(), 'relate': relate, 'intersect': intersect, 'equal': equal, 'count': count, 'if': if, 'assign': assign, 'forall': forall, 'and': and, 'or': or, 'true': true, 'false': false, 'Pr': Pr, 'color': color, 'shape': shape, 'is-red': is-red, 'is-blue': is-blue, 'is-ship': is-ship, 'is-house': is-house, 'is-green-f': is-green-f, 'is-good-f': is-good-f, 'is-good-g': is-good-g, 'new_func': new_func}
shape:[5, 7] max:1.52438223361969 min:-4.282702922821045
shape:[5, 7] max:-0.4794377088546753 min:-4.314482688903809


In [ ]:
that is to say...